Name: Sena Ezgi Anadollu

ID: 201401017

Course: YAP441/BIL541

# Mastermind Çözücü

Beklenen Çıktılar:
Her adımda codebreaker algoritmanın nasıl bir tahminde bulunduğunun açıkça görülmesi gerekiyor. 
•	En iyi tahmin yöntemi ve ortalama tahmin sayısı karşılaştırmaları.
•	Her yöntemin başarı yüzdesi,puanı ve verimlilik analizi.

Sistemin Çalışma Şekli:
•	Kullanıcılar  gizli renk kombinasyonunu terminal veya komut satırı arayüzü üzerinden girebilecek ve çözüme ulaşabilecektir.
•	Kullanıcı giriş yapmadığında, sistem otomatik olarak rastgele bir kombinasyon oluşturacak ve çözecektir.
Hamle sayısına göre de puan hesaplanır


In [63]:
import random
import random
import math
import time
import itertools

Parametreler


In [64]:
colors = ['R', 'G', 'B', 'Y', 'O', 'P']
code_length = 4
population_size = 100
max_feedback_calls = 10  # get_feedback fonksiyonu en fazla 10 defa çağrılabilir.
tournament_size = 5
mutation_rate = 0.1

Yardımcı Fonksiyonlar

In [65]:
def generate_individual():
    """Rastgele bir tahmin (genetik algoritma için birey) oluşturur."""
    return [random.choice(colors) for _ in range(code_length)]

In [66]:
def simulated_feedback(secret, guess):
    """
    Gizli kod ile guess arasındaki simüle edilmiş geri bildirimi (siyah, beyaz)
    hesaplar.
    Bu fonksiyon, geçmiş tahminlerin geri bildirimlerine benzer tahminler yapabilmek için kullanılır.
    """
    black = sum(s == g for s, g in zip(secret, guess))
    secret_counts = {color: secret.count(color) for color in colors}
    guess_counts  = {color: guess.count(color) for color in colors}
    common = sum(min(secret_counts[color], guess_counts.get(color, 0)) for color in colors)
    white = common - black
    return (black, white)

In [67]:
def is_consistent(candidate, history):
    """
    'history' içindeki her (tahmin, geri bildirim) çifti için,
    'candidate' tahmini ile simüle edilen geri bildirimin aynı olup olmadığını kontrol eder.
    """
    for past_guess, past_fb in history:
        if simulated_feedback(candidate, past_guess) != past_fb:
            return False
    return True


In [68]:

def candidate_fitness(candidate, history):
    """
    Aday tahminin geçmiş geri bildirimlere uyumunu ölçer.
    Her uyumlu geri bildirim için 1 puan eklenir.
    """
    score = 0
    for past_guess, past_fb in history:
        if simulated_feedback(candidate, past_guess) == past_fb:
            score += 1
    return score


In [69]:

def tournament_selection(population, history):
    """
    Popülasyondan turnuva seçimi yaparak, en yüksek fitness'e sahip bireyi seçer.
    """
    sample_size = min(tournament_size, len(population))
    tournament = random.sample(population, sample_size)
    tournament.sort(key=lambda cand: candidate_fitness(cand, history), reverse=True)
    return tournament[0]



In [70]:
def crossover(parent1, parent2):
    """
    Tek noktalı çaprazlama uygular. Parent1'in başı + Parent2'nin sonu
    """
    if code_length < 2:
        return parent1[:]
    crossover_point = random.randint(1, code_length - 1)
    return parent1[:crossover_point] + parent2[crossover_point:]



In [71]:
def mutate(individual):
    """
    Genetik çeşitliliği artırmak için belirlenen mutasyon oranına göre, bireyin her bir geninde rastgele değişiklik yapar.
    """
    new_ind = individual[:]
    for i in range(code_length):
        if random.random() < mutation_rate:
            current = new_ind[i]
            new_color = random.choice(colors)
            while new_color == current:
                new_color = random.choice(colors)
            new_ind[i] = new_color
    return new_ind

## Mastermind Oyun Sınıfı 

In [72]:
class MastermindGame:
    """
    Gizli kodu saklar ve tahminlere geri bildirim üretir.
    """
    def __init__(self, secret=None):
        self.code_length = code_length
        self.colors = colors
        self.secret = secret if secret is not None else generate_individual()
        self.feedback_calls = 0
        self.max_feedback_calls = max_feedback_calls

    def get_feedback(self, guess):
        """
        Gerçek geri bildirimi üretir. 10 tahmin hakkı aşıldığında hata fırlatır.
        """
        if self.feedback_calls >= self.max_feedback_calls:
            raise Exception("Maximum feedback calls reached (10 tahmin hakkı tükendi).")
        self.feedback_calls += 1
        black = sum(s == g for s, g in zip(self.secret, guess))
        secret_counts = {color: self.secret.count(color) for color in colors}
        guess_counts  = {color: guess.count(color) for color in colors}
        common = sum(min(secret_counts[color], guess_counts.get(color, 0)) for color in colors)
        white = common - black
        return (black, white)


## Algoritmalar

### 1. Genetik Algoritma

In [73]:
def genetic_algorithm(game):
    history = []  # (tahmin, geri bildirim) çiftleri
    population = [generate_individual() for _ in range(population_size)]
    generation = 1
    best_guess = None

    while generation <= max_feedback_calls:
        if not population:
            population = [generate_individual() for _ in range(population_size)]
            population = [cand for cand in population if is_consistent(cand, history)]
        
        best_candidate = max(population, key=lambda cand: candidate_fitness(cand, history))
        try:
            feedback = game.get_feedback(best_candidate)
        except Exception as e:
            print(e)
            break
        history.append((best_candidate, feedback))
        best_guess = best_candidate
        
        if feedback[0] == code_length:
            break

        population = [cand for cand in population if is_consistent(cand, history)]
        if not population:
            population = []
            while len(population) < population_size:
                cand = generate_individual()
                if is_consistent(cand, history):
                    population.append(cand)
        
        new_population = []
        while len(new_population) < population_size:
            parent1 = tournament_selection(population, history)
            parent2 = tournament_selection(population, history)
            child = crossover(parent1, parent2)
            child = mutate(child)
            if is_consistent(child, history):
                new_population.append(child)
        population = new_population
        generation += 1

    return history, generation, best_guess

In [74]:
def solve_with_genetic_algorithm(secret=None):
    game = MastermindGame(secret=secret)
    history, generations, best_guess = genetic_algorithm(game)
    return history, generations, best_guess, game.secret

### 2. Constraint Propagation (Kısıt Yayılımı)

In [75]:
def solve_with_constraint_propagation(secret=None):
    game = MastermindGame(secret=secret)
    possible_codes = [list(code) for code in itertools.product(colors, repeat=code_length)]
    history = []
    
    # İlk tahmin: rastgele veya sabit seçim
    guess = random.choice(possible_codes)
    
    while len(history) <= max_feedback_calls:
        feedback = game.get_feedback(guess)
        history.append((guess, feedback))
        if feedback[0] == code_length:
            return history, len(history), guess, game.secret
        
        # Geri bildirimle uyumlu kodları filtrele
        possible_codes = [code for code in possible_codes if simulated_feedback(code, guess) == feedback]
        if possible_codes:
            guess = possible_codes[0]
        else:
            break
    return history, len(history), guess, game.secret

### 3. Minimax Algoritması

In [76]:
def minimax_guess(possible_codes):

    if len(possible_codes) == 1:
        return possible_codes[0]
    best_guess = None
    best_score = float('inf')
    possible_feedbacks = [(b, w) for b in range(code_length+1) for w in range(code_length+1-b)]
    
    for guess in possible_codes:
        worst_case = 0
        for fb in possible_feedbacks:
            count = sum(1 for code in possible_codes if simulated_feedback(guess, code) == fb)
            worst_case = max(worst_case, count)
        if worst_case < best_score:
            best_score = worst_case
            best_guess = guess
    return best_guess


In [77]:
def solve_with_minimax(secret=None):
    game = MastermindGame(secret=secret)
    possible_codes = [list(code) for code in itertools.product(colors, repeat=code_length)]
    history = []
    initial_guess = ['R','R','G','G'] if ['R','R','G','G'] in possible_codes else possible_codes[0]
    guess = initial_guess
    
    while len(history) <= max_feedback_calls:
        feedback = game.get_feedback(guess)
        history.append((guess, feedback))
        if feedback[0] == code_length:
            return history, len(history), guess, game.secret
        
        possible_codes = [code for code in possible_codes if simulated_feedback(guess, code) == feedback]
        if not possible_codes:
            break
        guess = minimax_guess(possible_codes)
    return history, len(history), guess, game.secret

### 4. Simulated Annealing

In [78]:
def cost(candidate, history):
    """
    History'deki her tahmin için, simüle edilen geri bildirim ile alınan geri bildirim arasındaki
    farkın toplamını maliyet olarak hesaplar.
    """
    total = 0
    for past_guess, past_fb in history:
        sim_fb = simulated_feedback(candidate, past_guess)
        total += abs(sim_fb[0] - past_fb[0]) + abs(sim_fb[1] - past_fb[1])
    return total


In [79]:

def simulated_annealing_search(history, iterations=1000, initial_temp=100.0, cooling_rate=0.95):
    """
    History'ye göre maliyeti minimize eden bir tahmini, simulated annealing yöntemiyle arar.
    """
    current = generate_individual()
    current_cost = cost(current, history)
    best = current[:]
    best_cost = current_cost
    T = initial_temp
    for _ in range(iterations):
        if current_cost == 0:
            return current
        # Komşu çözüm üret: Rastgele bir pozisyonda renk değiştir.
        neighbor = current[:]
        i = random.randint(0, code_length - 1)
        neighbor[i] = random.choice(colors)
        neighbor_cost = cost(neighbor, history)
        delta = neighbor_cost - current_cost
        if delta < 0 or random.random() < math.exp(-delta / T):
            current = neighbor
            current_cost = neighbor_cost
            if current_cost < best_cost:
                best = current[:]
                best_cost = current_cost
        T *= cooling_rate
        if T < 1e-3:
            break
    return best


In [80]:

def solve_with_simulated_annealing(secret=None):
    game = MastermindGame(secret=secret)
    history = []
    while len(history) < max_feedback_calls:
        # History'ye göre simulated annealing ile uygun tahmini bul.
        candidate = simulated_annealing_search(history)
        feedback = game.get_feedback(candidate)
        history.append((candidate, feedback))
        if feedback[0] == code_length:
            return history, len(history), candidate, game.secret
    return history, len(history), candidate, game.secret

## Ana Program Akışı

In [83]:
def main():
    # Kullanıcıdan gizli kod oluşturulması istenir.
    secret_input = input("4 haneli gizli renk kombinasyonunu sağdaki renkleri kullanarak oluşturun (R, G, B, Y, O, P) veya rastgele bir kombinasyon oluşturmak için 'S' tuşuna basın: ")
    secret_input = secret_input.strip().upper()
    
    if secret_input == 'S':
        secret_code = generate_individual()
    elif len(secret_input) == 4 and all(c in colors for c in secret_input):
        secret_code = list(secret_input)
    else:
        print("Geçersiz giriş, rastgele gizli kod oluşturuluyor.")
        secret_code = generate_individual()
    
    print(f"\nGizli Kod: {tuple(secret_code)}\n")
    
    print("Algoritma Seçiniz:")
    print("1. Genetik Algoritma")
    print("2. Constraint Propagation")
    print("3. Minimax Algoritma")
    print("4. Simulated Annealing")
    choice = input("Seçiminiz (1/2/3/4): ")
    
    if choice == "1":
        history, generations, best_guess, secret = solve_with_genetic_algorithm(secret_code)
    elif choice == "2":
        history, generations, best_guess, secret = solve_with_constraint_propagation(secret_code)
    elif choice == "3":
        history, generations, best_guess, secret = solve_with_minimax(secret_code)
    elif choice == "4":
        history, generations, best_guess, secret = solve_with_simulated_annealing(secret_code)
    else:
        print("Geçersiz seçim, varsayılan olarak Genetik Algoritma seçildi.\n")
        history, generations, best_guess, secret = solve_with_genetic_algorithm(secret_code)
    
    print("\nTahmin Adımları:")
    for i, (guess, feedback) in enumerate(history, start=1):
        print(f"{i}. Tahmin: {tuple(guess)} -> Siyah Taş: {feedback[0]}, Beyaz Taş: {feedback[1]}")
    
    print(f"\nÇözüm Bulundu: {tuple(best_guess)}")
    print(f"Hamle Sayısı: {len(history)}")
    score = (max_feedback_calls - len(history)) * 10
    print(f"Puan: {score}")

if __name__ == "__main__":
    main()


Geçersiz giriş, rastgele gizli kod oluşturuluyor.

Gizli Kod: ('O', 'B', 'Y', 'G')

Algoritma Seçiniz:
1. Genetik Algoritma
2. Constraint Propagation
3. Minimax Algoritma
4. Simulated Annealing

Tahmin Adımları:
1. Tahmin: ('G', 'Y', 'R', 'Y') -> Siyah Taş: 0, Beyaz Taş: 2
2. Tahmin: ('Y', 'P', 'G', 'B') -> Siyah Taş: 0, Beyaz Taş: 3
3. Tahmin: ('P', 'G', 'Y', 'O') -> Siyah Taş: 1, Beyaz Taş: 2
4. Tahmin: ('B', 'O', 'Y', 'G') -> Siyah Taş: 2, Beyaz Taş: 2
5. Tahmin: ('P', 'O', 'B', 'G') -> Siyah Taş: 1, Beyaz Taş: 2
6. Tahmin: ('B', 'R', 'P', 'G') -> Siyah Taş: 1, Beyaz Taş: 1
7. Tahmin: ('O', 'P', 'Y', 'G') -> Siyah Taş: 3, Beyaz Taş: 0
8. Tahmin: ('O', 'B', 'Y', 'G') -> Siyah Taş: 4, Beyaz Taş: 0

Çözüm Bulundu: ('O', 'B', 'Y', 'G')
Hamle Sayısı: 8
Puan: 20


## Deney ve Karşılaştırma

1000 Senaryo Üzerinde Test

Kodun bu kısmı zaman verimliliği açısından google colab'da çalıştırılmıştır, elde edilen çıktılar rapora ve sunuma eklenmiştir.

In [ ]:

algorithms = {
    "Genetik Algoritma": solve_with_genetic_algorithm,
    "Simulated Annealing": solve_with_simulated_annealing,
    "Minimax": solve_with_minimax,
    "Constraint Propagation": solve_with_constraint_propagation
}

for name, solver in algorithms.items():
    results = []
    success = []
    total_start_time = time.time()

    for _ in range(1000):

        history, generations, best_guess, secret = solver()
        results.append(generations)
        if best_guess == secret:
          success.append(1)
        else:
          success.append(0)


    total_end_time = time.time()

    print(f"\n{name} Sonuçları:")
    print(f"Ortalama Tahmin Sayısı: {sum(results) / len(results):.2f}")
    success_count = sum(success)
    success_rate = (success_count / len(success)) * 100
    print(f"{len(results)} çalıştırmada başarı yüzdesi: %{success_rate:.2f}")
    print(f"Minimum hamle sayısı: {min(results)}")
    print(f"Maksimum hamle sayısı: {max(results)}")
    print(f"Toplam süre: {total_end_time - total_start_time:.2f} saniye")



Genetik Algoritma Sonuçları:
Ortalama Tahmin Sayısı: 4.60
10 çalıştırmada başarı yüzdesi: %100.00
Minimum hamle sayısı: 4
Maksimum hamle sayısı: 6
Toplam süre: 3.02 saniye

Simulated Annealing Sonuçları:
Ortalama Tahmin Sayısı: 5.30
10 çalıştırmada başarı yüzdesi: %100.00
Minimum hamle sayısı: 5
Maksimum hamle sayısı: 6
Toplam süre: 0.07 saniye

Minimax Sonuçları:
Ortalama Tahmin Sayısı: 4.60
10 çalıştırmada başarı yüzdesi: %100.00
Minimum hamle sayısı: 4
Maksimum hamle sayısı: 5
Toplam süre: 38.87 saniye

Constraint Propagation Sonuçları:
Ortalama Tahmin Sayısı: 4.90
10 çalıştırmada başarı yüzdesi: %100.00
Minimum hamle sayısı: 4
Maksimum hamle sayısı: 7
Toplam süre: 0.07 saniye


## Sonuçlar
Constraint Propagation: Genellikle en hızlı ve etkili yöntemdir.

Minimax: Daha uzun sürebilir ancak optimal sonuçlar verir. 6 adımda çözmeyi garanti eder.

Genetik Algoritma: Büyük popülasyonlarda iyi sonuçlar verebilir, ancak rastgelelik nedeniyle tutarsız olabilir.

Simulated Annealing: Oldukça hızlı sonuca ulaşsa da %100 doğru sonucu bulma garantisi yoktur.


## Geliştirme ve İyileştirme
Algoritmaların performansını artırmak için parametreleri optimize edildi.

Kullanıcı arayüzü ekleyerek oyunu interaktif hale getirilebilir.